In [1]:
!pip3 install torchgan

     |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
# Pytorch and Torchvision Imports
import torch
import torchvision
from torch.optim import Adam
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutils
# Torchgan Imports
from torchgan.losses import MinimaxGeneratorLoss, MinimaxDiscriminatorLoss
import torchgan
from torchgan.layers import SpectralNorm2d, SelfAttention2d, ResidualBlock2d
from torchgan.models import Generator, Discriminator
from torchgan.losses import WassersteinGeneratorLoss, WassersteinDiscriminatorLoss, WassersteinGradientPenalty
from torchgan.trainer import Trainer

from google.colab.patches import cv2_imshow
import cv2


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def apple_loader(data_size=512):
  for root, dirs, files in os.walk("../content/drive/My Drive/apples/images/"):
      res = np.zeros((data_size, 3,64,64))
      ex_count = 0
      for i, file in enumerate(files):
        
     
        if file.endswith(".jpg"):
             image=cv2.imread(root+file)
             try:
                image.resize(3,64,64)
                image = image/ 127.5 - 1.
                res[i,:,:,:] = image
             except AttributeError:
                ex_count+=1
                pass
             
        if i > data_size-2 : 
          print(ex_count)
          print(len(files))
          features = torch.from_numpy(res)
          features = features.float()
          targets = torch.ones(data_size)
          train_tensor = data.TensorDataset(features, targets)
          train_loader = data.DataLoader(train_tensor, batch_size=data_size//4, shuffle=True)
          return train_loader

In [19]:
for i in range(1,6):
  path = 'images/epoch' + str(i) + '_generator.png'
  print(path)
  sample = cv2.imread(path)
  cv2_imshow(sample)

images/epoch1_generator.png


images/epoch2_generator.png


images/epoch3_generator.png


images/epoch4_generator.png


images/epoch5_generator.png


In [0]:
sample = cv2.imread('images/epoch13_generator.png')
sample.shape

(138, 274, 3)

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    # Use deterministic cudnn algorithms
    torch.backends.cudnn.deterministic = True
    epochs = 20
else:
    device = torch.device("cpu")
    epochs = 10

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))

Device: cuda:0
Epochs: 20


In [0]:
class ResUp(nn.Module):
  def __init__(self, input_channels, output_channels):
    super(ResUp, self).__init__()
    self.skip = nn.Sequential(
                  nn.Upsample(2),
                  nn.Conv2d(input_channels , output_channels, 1))
    self.model = nn.Sequential(
                nn.BatchNorm2d(input_channels), nn.LeakyReLU(0.2),
                nn.Upsample(2),
                nn.Conv2d(input_channels, output_channels , 3,1,1),
                nn.BatchNorm2d(output_channels), nn.LeakyReLU(0.2),
                nn.Conv2d(output_channels, output_channels, 3,1,1))
  def forward(self, x):
    main = self.model(x)
    skip = self.skip(x)
    out = torch.add(main, skip)
    return out


In [0]:
class ResDown(nn.Module):
  def __init__(self, input_channels, output_channels):
    super(ResDown, self).__init__()
   
    self.skip = nn.Sequential(
                  nn.Conv2d(input_channels, output_channels, 1),
                  nn.AvgPool2d(2))
    self.model = nn.Sequential(
                nn.LeakyReLU(0.2),
                nn.Conv2d(input_channels, output_channels, 3,1,1),
                nn.LeakyReLU(0.2),
                nn.Conv2d(output_channels, output_channels, 3,1,1),
                nn.AvgPool2d(2))
  def forward(self, x):
    main = self.model(x)
    skip = self.skip(x)
    out = torch.add(skip, main)
    return out

In [0]:
class BigGanDiscriminator(Discriminator):
    def __init__(self, chn=64):
        super(BigGanDiscriminator, self).__init__(3, 'none')
        self.pre_conv = nn.Sequential(SpectralNorm2d(nn.Conv2d(3, chn, 3,1,1)),
                                      nn.ReLU(),
                                      SpectralNorm2d(nn.Conv2d(chn, chn, 3,1,1)),
                                      nn.AvgPool2d(2))
        self.pre_skip = SpectralNorm2d(nn.Conv2d(3, chn, 1))

        self.conv = nn.Sequential(ResDown(chn, chn),
                                  SelfAttention2d(chn),
                                  ResDown(chn, 2*chn),    
                                  ResDown(2*chn, 4*chn),
                                  ResDown(4*chn, 8*chn),
                                  ResDown(8*chn, 16*chn))
        self.res_layer = ResidualBlock2d([16*chn, 16*chn], [3], [1],[1])
        self.linear = SpectralNorm2d(nn.Linear(16*chn, 1))

        self.last = nn.LeakyReLU(0.2)
    def forward(self, x):
      out = self.pre_conv(x)
      out = out + self.pre_skip(F.avg_pool2d(x, 2))
      out = self.conv(out)
      out = self.res_layer(out)
      out = out.view(out.size(0), out.size(1), -1)
      out = out.sum(2)
      out = self.linear(out)
      out = self.last(out)
      
      return out

In [0]:
class BigGanGenerator(Generator):
    def __init__(self, chn=64):
        super(BigGanGenerator, self).__init__(1, 'none')
        self.linear = SpectralNorm2d(nn.Linear(1, 4 * 4 * 16 * chn))
        self.conv = nn.Sequential(ResUp(16*chn, 16*chn),
                                ResUp(16*chn, 8*chn),
                                ResUp(8*chn, 4*chn),
                                ResUp(4*chn, 2*chn),
                                SelfAttention2d(2*chn),
                                ResUp(2*chn, chn))
        self.last = nn.Sequential(nn.BatchNorm2d(chn), nn.LeakyReLU(0.2),
                nn.Conv2d(chn, 3, 1), nn.Tanh())
    def forward(self, x):
        out = self.linear(x)
        out = out.view(-1, 1024, 4, 4)
        out = self.conv(out)
        out = self.last(out)
        return out

In [0]:
network_params = {
    "generator": {"name": BigGanGenerator, "args": {"chn": 64},
                  "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}}},
    "discriminator": {"name": BigGanDiscriminator, "args": {"chn": 64},
                      "optimizer": {"name": Adam, "args": {"lr": 0.0004, "betas": (0.5, 0.999)}}}
}

In [0]:
trainer = Trainer(network_params, [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()], sample_size=32, epochs=100, device=device)

In [18]:
trainer(apple_loader(1100))

2
1162
Saving Model at './model/gan0.model'
Epoch 1 Summary
generator Mean Gradients : 30.765573479183285
discriminator Mean Gradients : 11.530714539356989
Mean Running Discriminator Loss : 0.9945204108953476
Mean Running Generator Loss : 0.7149397879838943
Generating and Saving Images to ./images/epoch1_generator.png

Saving Model at './model/gan1.model'
Epoch 2 Summary
generator Mean Gradients : 18.175710927322523
discriminator Mean Gradients : 6.5400194688680635
Mean Running Discriminator Loss : 0.7687687538564205
Mean Running Generator Loss : 0.7978639602661133
Generating and Saving Images to ./images/epoch2_generator.png

Saving Model at './model/gan2.model'
Epoch 3 Summary
generator Mean Gradients : 12.66346503662194
discriminator Mean Gradients : 4.610506668646183
Mean Running Discriminator Loss : 0.5670338794589043
Mean Running Generator Loss : 1.2670901815096538
Generating and Saving Images to ./images/epoch3_generator.png

Saving Model at './model/gan3.model'
Epoch 4 Summary


KeyboardInterrupt: ignored